# 1 Authenticate at the Twitter API

In [51]:
import pandas as pd
import numpy as np
import json
import requests
import datetime as dt
import logging
import threading
import time
from __future__ import print_function

import tweepy 
import gate_api
from gate_api.exceptions import ApiException, GateApiException
from twitter_secrets import twitter_secrets as ts # place the twitter_secrets file under <User>/anaconda3/Lib

consumer_key = ts.CONSUMER_KEY
consumer_secret = ts.CONSUMER_SECRET
access_token = ts.ACCESS_TOKEN
access_secret = ts.ACCESS_SECRET

### Print API Auth Data (leave disabled for security reasons)
# print(f'consumer_key: {consumer_key}')
# print(f'consumer_secret: {consumer_secret}')
# print(f'access_token: {access_token}')
# print(f'access_secret: {access_token}')

#authenticating to access the twitter API
auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth)

# 2 Define Twitter Tweets 

In [52]:
def send_volume_tweet(v):
    tweet='this is an automated test tweet using #Python $BTC $ETH'
    api.update_status(tweet)

def send_pricechange_tweet(v):
    tweet='this is an automated test tweet using #Python $BTC $ETH'
    api.update_status(tweet)

# 3 Get Price Data from Gate.io

In [53]:
class Prices:
    """Class that uses the gate api to retrieve currency data."""

    def __init__(self, config):
        self._config = config
        self._logger = logging.getLogger(__name__)
        configuration = gate_api.Configuration(host="https://api.gateio.ws/api/v4")
        api_client = gate_api.ApiClient(configuration)
        self._api_instance = gate_api.SpotApi(api_client)
        self._price_history = {}
        self._cont_update_thread = None
        self._stop_cont_update_thread = None
        self._price_history_lock = threading.Lock()

    def get_price_history(self):
        """Returns a dictionary with the price histories for the currencies."""
        return self._price_history, self._price_history_lock

    def get_latest_prices(self):
        """Gets new price data and adds the values to a DataFrame.

        Returns the DataFrame in a dictionary with the currencies as keys."""
        timestamp = dt.datetime.now()
        try:
            api_response = self._api_instance.list_tickers()
        except GateApiException as e:
            logging.warning(
                "Gate api exception, label: %s, message: %s\n" % (e.label, e.message)
            )
            return {}
        except ApiException as e:
            logging.warning("Exception when calling SpotApi->list_tickers: %s\n" % e)
            return {}
        latest_prices = {}
        for response in api_response:
            currency = response.currency_pair
            if "USDT" not in currency or "BEAR" in currency:
                continue
            value_dict = {
                "base_volume": pd.to_numeric(response.base_volume),
                "change_percentage": pd.to_numeric(response.change_percentage),
                "etf_leverage": pd.to_numeric(response.etf_leverage),
                "etf_net_value": pd.to_numeric(response.etf_net_value),
                "etf_pre_net_value": pd.to_numeric(response.etf_pre_net_value),
                "etf_pre_timestamp": response.etf_pre_timestamp,
                "high_24h": pd.to_numeric(response.high_24h),
                "highest_bid": pd.to_numeric(response.highest_bid),
                "high_bid": pd.to_numeric(response.highest_bid),
                "last": pd.to_numeric(response.last),
                "low_24h": pd.to_numeric(response.low_24h),
                "lowest_ask": pd.to_numeric(response.lowest_ask),
                "quote_volume": pd.to_numeric(response.quote_volume),
                "timestamp": timestamp,
            }
            latest_prices[currency] = pd.DataFrame(value_dict, index=[1])
        return latest_prices

    def start_cont_update(self):
        self._stop_cont_update_thread = threading.Event()
        self._stop_cont_update_thread.clear()
        self._cont_update_thread = threading.Thread(
            target=self._cont_update,
            args=(
                self._stop_cont_update_thread,
                self._price_history_lock,
            ),
        )
        self._cont_update_thread.start()
        self._logger.info("Started continuous price logging")

    def _cont_update(self, stop_event, lock):
        """Continuously adds new prices to the price history."""
        while not stop_event.is_set():
            start_time = time.time()
            lock.acquire()
            for currency, df in self.get_latest_prices().items():
                if currency in self._price_history.keys():
                    self._price_history[currency] = self._price_history[
                        currency
                    ].append(df, ignore_index=True)
                else:
                    self._price_history[currency] = df
            lock.release()
            self._logger.debug("Currency_dfs updated")
            self._wait_before_update(start_time)

    def _wait_before_update(self, start_time):
        elapsed_time = time.time() - start_time
        self._logger.debug(f"Elapsed time: {elapsed_time}")
        if elapsed_time > self._config["price_update_delay"]:
            delay = 0
            self._logger.warning(
                #"It took longer to retrieve the price data than the update_delay!"
            )
        else:
            delay = self._config["price_update_delay"] - elapsed_time
        self._logger.debug(f"Waiting {delay}s until next update")
        time.sleep(delay)


# 4 Calculate Indicator Values

In [54]:
def calc_indicators(price_history):
    indicators = {}
    indicators_over_all = calc_indicators_over_all(price_history)
    for currency, df in price_history.items():
        if len(df) <= 2:
            logging.getLogger().debug(
                f"Skipped '{currency} when calculating indicators due to a lack of information"
            )
            continue
        volume = df["base_volume"].iloc[-1]
        last_price = df["last"].iloc[-1]
        moving_avg_price = df["last"].mean()
        moving_average_volume = df["base_volume"].mean()
        moving_average_deviation_percent = np.round(
            div(last_price, moving_avg_price) - 1, 2
        )

        price_before = df["last"].iloc[-2]
        price_delta = last_price - price_before
        price_delta_p = div(price_delta, last_price)
        price_delta_before = price_before - df["last"].iloc[-3]
        price_delta_p_before = div((price_before - df["last"].iloc[-3]), price_before)
        low_24h = df["low_24h"].iloc[-1]
        high_24h = df["high_24h"].iloc[-1]
        low_high_diff_p = div(high_24h - low_24h, low_24h)
        change_percentage = df["change_percentage"].iloc[-1]

        indicator_values = {
            "last_price": last_price,
            "price_before": price_before,
            "volume": volume,
            "moving_avg_price": moving_avg_price,
            "moving_average_volume": moving_average_volume,
            "moving_average_deviation_percent": moving_average_deviation_percent,
            "price_delta_p": price_delta_p,
            "price_delta": price_delta,
            "price_delta_before": price_delta_before,
            "price_delta_p_before": price_delta_p_before,
            "high_24h": high_24h,
            "low_24h": low_24h,
            "low_high_diff_p": low_high_diff_p,
            "change_percentage": change_percentage,
        }
        indicator_values.update(indicators_over_all)
        indicators[currency] = indicator_values
    return indicators


def calc_indicators_over_all(price_history):
    avg_change_p = 0
    for currency, df in price_history.items():
        avg_change_p += df["change_percentage"].iloc[-1]
    nr_of_currencies = len(price_history)
    avg_change_p = div(avg_change_p, nr_of_currencies)
    values = {
        "avg_change_p": avg_change_p,
    }
    return values


def div(dividend, divisor, alt_value=0.0):
    return dividend / divisor if divisor != 0 else alt_value

# 4 Signaling Logic

In [55]:
def check_signal(currency, indicators, cs_config):
    ind = indicators[currency]
    signal = ''
    if (
            ind['moving_avg_price'] > 0
            and ind['last_price'] > 0.0
            and abs(ind['price_delta_p']) >= cs_config["delta_threshold_p"]
            and ind['volume'] > 0
            and ind['price_delta'] > 0.0
    ):
        # up
        if ind['price_delta_p'] >= cs_config["moving_avg_threshold_up_p"]:
            if ind['price_delta_p_before'] >= 0:
                if ind['price_delta_p_before'] < cs_config["delta_threshold_p"] \
                        and ind['price_delta_p_before'] < ind['price_delta_p'] * 1.1:
                    movement_type = 'up +'
                elif ind['price_delta_p_before'] < ind['price_delta_p'] * 1.1:
                    movement_type = 'up and accelerates ++'
        # down
        elif ind['price_delta_p'] >= cs_config["moving_avg_threshold_down_p"] and ind['price_delta'] < 0:
            if ind['price_delta_p_before'] < 0:
                if ind['price_delta_p_before'] < cs_config["delta_threshold_p"]:
                    movement_type = 'initial_down -'
                elif ind['price_delta_p_before'] < ind['price_delta_p'] * 1.1:
                    movement_type = 'down further accelerates --'
            else:
                movement_type = 'retracing to ' + str(ind['price_before']) + ' +'

            
        # check for 24 h low or high
        if ind['last_price'] < ind['low_24h']:
            print(currency + ' new 24h low ' + str(ind['last_price']))
        elif ind['last_price'] > ind['high_24h']:
            print(currency + ' new 24h high ' + str(ind['last_price']))
        signal = get_signal_log(movement_type, currency, ind['price_delta_p'], ind['last_price'],
                                        ind['moving_avg_price'], ind['volume'], ind['price_delta'], ind['change_percentage'],
                                    ind['high_24h'], ind['low_24h'], ind['low_high_diff_p'])
    return signal


def get_signal_log(movement_type, currency, price_delta_p, last_price, moving_avg_price, volume, price_delta, daily_up_p, high_24h, low_24h, low_high_diff_p):
    signal = f'{currency} {movement_type}'\
        f'{np.round(price_delta_p * 100, 2)}% ' \
        f' MA: ${np.round(moving_avg_price, 5)} ' \
        f' last_price: ${np.round(last_price, 5)} ' \
        f' price delta: {np.round(price_delta, 5)} ' \
        f' volume: ${np.round(volume, 1)}' \
        f' daily change {np.round(daily_up_p, 2)}%' \
        f' high_24h: ${high_24h}' \
        f' low_24h: ${low_24h}' \
        f' low_high_diff_p: ${np.round(low_high_diff_p * 100, 2)}%'
    return signal


def trigger_buy(currency, last_price):
    print('bought ' + currency + 'at ' + last_price)

def trigger_sell(currency, last_price):
    print('bought ' + currency + 'at ' + last_price)

def stop_buy(currency, last_price):
    print('bought ' + currency + 'at ' + last_price)

def stop_loss(currency, last_price):
    print('bought ' + currency + 'at ' + last_price)

# 5 Start Streaming Data

In [ ]:
RUNS = 1000
CYCLE_DELAY = 20
EVAL_PRICES_DELAY = 50
CURRENCY_PAIR = ""
PRICES_CONFIG = {"price_update_delay": 20}

CHECK_SIGNAL_CONFIG = {
    "moving_avg_threshold_down_p": 0.05,
    "moving_avg_threshold_up_p": 0.05,
    "delta_threshold_p": 0.03,
    'enable_twitter': True,
}

if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO, format="\033[02m%(asctime)s %(levelname)s: %(message)s"
    )
    logger = logging.getLogger(__name__)
    prices = Prices(PRICES_CONFIG)
    prices.start_cont_update()
    currency_dfs = {}
    logging.info(f"Crypto bot is starting - please wait")
    logger.info(f"Collecting crypto data from gate.io for {EVAL_PRICES_DELAY}s")
    time.sleep(EVAL_PRICES_DELAY)
    logger.info(f"\n<< Crypto lobster signal bot started :-) >>")
    logger.info(f"<< Checking prices every {CYCLE_DELAY} seconds >>")
    logger.info(f"Now checking for signals - please wait\n")
    for i in range(RUNS):
        price_history, lock = prices.get_price_history()
        lock.acquire()
        indicators = calc_indicators(price_history)
        lock.release()
        for currency in indicators.keys():
            if not indicators[currency]:
                continue
            signal = check_signal(
                currency,
                indicators,
                CHECK_SIGNAL_CONFIG,
            )
            if signal:
                logger.info(signal)
        time.sleep(CYCLE_DELAY)

2021-05-17 22:52:42,565 INFO: Started continuous price logging
2021-05-17 22:52:42,584 INFO: Crypto bot is starting - please wait
2021-05-17 22:52:42,595 INFO: Collecting crypto data from gate.io for 50s
2021-05-17 22:52:43,278 WARNING: It took longer to retrieve the price data than the update_delay!
2021-05-17 22:52:43,604 WARNING: It took longer to retrieve the price data than the update_delay!
2021-05-17 22:52:48,901 WARNING: It took longer to retrieve the price data than the update_delay!
